### Libraries

In [27]:
from typing import Union
import numpy as np
from qclib.gates.mcu import MCU
from qiskit import (QuantumCircuit,
                    QuantumRegister,
                    ClassicalRegister,
                    transpile)
from qiskit.visualization import (plot_histogram,
                                  circuit_drawer)
from qiskit_aer import AerSimulator
from qiskit_aer.noise import NoiseModel
from qiskit.providers.models import BackendConfiguration
from qiskit.providers.fake_provider import GenericBackendV2
# from joblib import Parallel, delayed
import matplotlib.pyplot as plt
import time

In [3]:
class Utilities:

    def __init__(self, operator, error):
        self.operator = operator
        self.error = error

    @staticmethod
    def pauli_matrices(string: str) -> Union[np.ndarray, None]:
        string_case = string.lower()
        match string_case:
            case "x":
                return np.array([
                    [0, 1],
                    [1, 0]
                ])
            case "y":
                return np.array([
                    [0, -1j],
                    [1j, 0]
                ])
            case "z":
                return np.array([
                    [1, 0],
                    [0, -1]
                ])
            case _:
                return None

    @staticmethod
    def transpose_conjugate(operator):
        return np.conjugate(np.transpose(operator))

    @staticmethod
    def pyramid_size(operator, error):
        mcu_dummy = MCU(operator, num_controls=100, error=error)
        # will be changed by mcu_dummy._get_n_base(x_dagger, error)
        return mcu_dummy._get_num_base_ctrl_qubits(operator, error)


In [38]:
def built_circuit(pauli_string='x', error=0.1, approximated = True):
    
    pauli_matrix = Utilities.pauli_matrices(pauli_string)
    pauli_matrix_dagger = Utilities.transpose_conjugate(pauli_matrix)
    n_base = Utilities.pyramid_size(pauli_matrix_dagger, error)

    controls = QuantumRegister(n_base, 'controls')
    target = QuantumRegister(1, 'target')
    classical = ClassicalRegister(1, "classic")

    circ = QuantumCircuit(controls, target, classical)
    for i in range(len(controls)):
        circ.x(i)
    circ.x(target)
    if approximated:
      MCU.mcu(circ, pauli_matrix_dagger, controls, target, error)
    else:
      MCU.mcu(circ, pauli_matrix_dagger, controls, target, 0)
    # circ.measure(target, [0])
    circ.measure_all()
    return circ

In [41]:
approx_circuit = built_circuit('x', 0.1)
print(approx_circuit)

            ┌───┐┌────────────┐ ░ ┌─┐                  
controls_0: ┤ X ├┤0           ├─░─┤M├──────────────────
            ├───┤│            │ ░ └╥┘┌─┐               
controls_1: ┤ X ├┤1           ├─░──╫─┤M├───────────────
            ├───┤│            │ ░  ║ └╥┘┌─┐            
controls_2: ┤ X ├┤2           ├─░──╫──╫─┤M├────────────
            ├───┤│            │ ░  ║  ║ └╥┘┌─┐         
controls_3: ┤ X ├┤3 Mcuapprox ├─░──╫──╫──╫─┤M├─────────
            ├───┤│            │ ░  ║  ║  ║ └╥┘┌─┐      
controls_4: ┤ X ├┤4           ├─░──╫──╫──╫──╫─┤M├──────
            ├───┤│            │ ░  ║  ║  ║  ║ └╥┘┌─┐   
controls_5: ┤ X ├┤5           ├─░──╫──╫──╫──╫──╫─┤M├───
            ├───┤│            │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
    target: ┤ X ├┤6           ├─░──╫──╫──╫──╫──╫──╫─┤M├
            └───┘└────────────┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
 classic: 1/═══════════════════════╬══╬══╬══╬══╬══╬══╬═
                                   ║  ║  ║  ║  ║  ║  ║ 
    meas: 7/═══════════════════════╩══╩══╩══╩══╩

In [42]:
real_circuit = built_circuit('x', 0.1, False)
print(real_circuit)

            ┌───┐┌────────┐ ░ ┌─┐                  
controls_0: ┤ X ├┤0       ├─░─┤M├──────────────────
            ├───┤│        │ ░ └╥┘┌─┐               
controls_1: ┤ X ├┤1       ├─░──╫─┤M├───────────────
            ├───┤│        │ ░  ║ └╥┘┌─┐            
controls_2: ┤ X ├┤2       ├─░──╫──╫─┤M├────────────
            ├───┤│        │ ░  ║  ║ └╥┘┌─┐         
controls_3: ┤ X ├┤3 Ldmcu ├─░──╫──╫──╫─┤M├─────────
            ├───┤│        │ ░  ║  ║  ║ └╥┘┌─┐      
controls_4: ┤ X ├┤4       ├─░──╫──╫──╫──╫─┤M├──────
            ├───┤│        │ ░  ║  ║  ║  ║ └╥┘┌─┐   
controls_5: ┤ X ├┤5       ├─░──╫──╫──╫──╫──╫─┤M├───
            ├───┤│        │ ░  ║  ║  ║  ║  ║ └╥┘┌─┐
    target: ┤ X ├┤6       ├─░──╫──╫──╫──╫──╫──╫─┤M├
            └───┘└────────┘ ░  ║  ║  ║  ║  ║  ║ └╥┘
 classic: 1/═══════════════════╬══╬══╬══╬══╬══╬══╬═
                               ║  ║  ║  ║  ║  ║  ║ 
    meas: 7/═══════════════════╩══╩══╩══╩══╩══╩══╩═
                               0  1  2  3  4  5  6 


In [43]:
def hist(circuit):
    # Generate a 5-qubit simulated backend
    
    size = len(circuit)
    backend = GenericBackendV2(num_qubits=size)
    
    # Transpile the ideal circuit to a circuit that can be directly executed by the backend
    transpiled_circuit = transpile(circuit, backend)
    transpiled_circuit.draw('mpl')
    
    # Run the transpiled circuit using the simulated backend
    job = backend.run(transpiled_circuit)
    counts = job.result().get_counts()
    return counts

In [45]:
h1 = hist(real_circuit)
h2 = hist(approx_circuit)